## Prepare example
[Example provided by Evidently](https://github.com/evidentlyai/evidently/blob/main/examples/sample_notebooks/evidently_test_presets.ipynb)

In [26]:
import numpy as np
from evidently.test_preset import DataDriftTestPreset
from evidently.test_suite import TestSuite
from sklearn import datasets


# Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(
    name="adult", version=2, as_frame="auto", parser="auto"
)
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]
adult_cur = adult[adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]

adult_cur.iloc[:2000, 3:5] = np.nan

# Run Evidently Tests
data_drift = TestSuite(
    tests=[
        DataDriftTestPreset(stattest="psi"),
    ]
)

data_drift.run(reference_data=adult_ref, current_data=adult_cur)

Alert Detected: True


In [27]:
# Save the report as HTML
from tools.helper_functions import get_evidently_html

html, html_bytes = get_evidently_html(data_drift)

## Is alert detected?

In [28]:
test_summary = data_drift.as_dict()
other_tests_summary = []

failed_tests = []
for test in test_summary["tests"]:
    if test["status"].lower() == "fail":
        failed_tests.append(test)

is_alert = any([failed_tests, other_tests_summary])
print(f"Alert Detected: {is_alert}")

Alert Detected: True


## Setup generic email details

In [ ]:
RECIPIENT_LIST = ["your.email@here.com"]
PROJECT_NAME = "Example Adult Education"
LINK_URL = "www.your-url.com"

## Send basic email

In [ ]:
# Note: uses ses.send_email function
from tools.send_email_basic import send_email_basic


if is_alert:
    send_email_basic(project_name=PROJECT_NAME, link_url=LINK_URL, recipient_list=RECIPIENT_LIST,)

## Send email attachment

In [ ]:
# Note: uses ses.send_raw_email function
# Note2: attachments must be read as binary
from tools.send_email_attachment import send_email_attachment, EmailAttachment


# Prepare email attachment
data_drift_report = EmailAttachment(file_name="data_drift.html", content=html_bytes)
attachment_list = [data_drift_report]

if is_alert:
    send_email_attachment(project_name=PROJECT_NAME, link_url=LINK_URL, recipient_list=RECIPIENT_LIST,
                          attachment_list=attachment_list)

## Send nice formatted email (Template)

In [ ]:
# Note: uses ses.send_email function
from tools.send_email_formatted import send_email_formatted


if is_alert:
    send_email_formatted(project_name=PROJECT_NAME, link_url=LINK_URL, recipient_list=RECIPIENT_LIST)